In [76]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score

from datetime import datetime

In [2]:
train = pd.read_csv('../data/train.csv')
students = pd.read_csv('../data/Student.csv')
internship = pd.read_csv('../data/Internship.csv')
test = pd.read_csv('../data/test.csv')

C:\Users\Abhishek\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Merge with internship details

In [45]:
train_with_internship = pd.merge(train, internship, how='left', on='Internship_ID')
test_with_internship = pd.merge(test, internship, how='left', on='Internship_ID')

In [46]:
train_with_internship_clean = train_with_internship.dropna(axis=1)
test_with_internship_clean = test_with_internship.dropna(axis=1)

In [47]:
categorical_features = train_wtih_internship_clean.select_dtypes(include=['object']).columns

In [48]:
for feature in categorical_features[1:]:
    lbl = LabelEncoder()
    feature_range = pd.concat([train_wtih_internship_clean[feature], test_with_internship_clean[feature]], axis=0)
    
    lbl.fit(feature_range)
    train_with_internship_clean.loc[:, feature] = lbl.transform(train_with_internship_clean[feature])
    test_with_internship_clean.loc[:, feature] = lbl.transform(test_with_internship_clean[feature])

C:\Users\Abhishek\Anaconda2\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [50]:
X = train_with_internship_clean[train_with_internship_clean.columns.drop(['Earliest_Start_Date', 'Is_Shortlisted'])]
y = train_wtih_internship_clean.Is_Shortlisted

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44) 

## Classifier

In [108]:
select = SelectKBest(chi2, k=30)
clf = RandomForestClassifier(n_estimators=300, criterion='entropy', n_jobs=-1)
# clf = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4)

pipeline = Pipeline([('select', select), ('clf', clf)])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
predsTrain = pipeline.predict_proba(X_train)[:, 1]
predsTest = pipeline.predict_proba(X_test)[:, 1]

In [ ]:
print 'AUC score on training set %f ' %(roc_auc_score(y_train, predsTrain))
print 'AUC score on test set %f ' %(roc_auc_score(y_test, predsTest))

## Train on full dataset

In [85]:
pipeline.fit(X, y)

Pipeline(steps=[('select', SelectKBest(k=50, score_func=<function chi2 at 0x000000001AC846D8>)), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

## Predict on the test set

In [90]:
test_features = test_with_internship_clean[test_with_internship_clean.columns.drop('Earliest_Start_Date')]

In [91]:
predictions = pipeline.predict_proba(test_features)[:, 1]

In [94]:
submission = pd.read_csv('../data/submission.csv')
submission['Internship_ID'] = test.Internship_ID
submission['Student_ID'] = test.Student_ID
submission['Is_Shortlisted'] = predictions

In [95]:
submission.to_csv('../submissions/encoded_labels_xgboose.csv', index=False)